# **Task : 대규모 도로명 주소 데이터에서 동대문구 행정동 추출하기**
1) 카카오 앱 화면에서 프로젝트명 등록 (https://developers.kakao.com/console/app)
2) 앱 > '플랫폼 키' 선택하여 REST API Key 발급
3) API를 사용하여 도로명 주소 -> 좌표(위도, 경도) -> 행정동 변환

--

In [1]:
pip install PyKakao --upgrade

In [4]:
from PyKakao import Local
api = Local(service_key = "???") #본인 api 키 입력

In [ ]:
import pandas as pd
hos = pd.read_excel('의료기관 목록.xlsx')

#세부 주소는 삭제하고 도로명주소만 전처리
hos["전처리_도로명주소"] = (
    hos["의료기관주소(도로명)"]
    .astype(str)
    .str.split(",")
    .str[0]
    .str.strip()
)

hos.head()

In [13]:
def addr_to_admin_dong(addr: str):
    # 1) 주소 → 좌표
    res = api.search_address(addr)
    docs = res.get("documents", [])
    if not docs:
        return None

    doc = docs[0]
    a = doc.get("address") or {}
    x = a.get("x")
    y = a.get("y")
    if not x or not y:
        return None

    # 2) 좌표 → 행정동
    reg = api.geo_coord2regioncode(x, y)
    rdocs = reg.get("documents", [])
    if not rdocs:
        return None

    # 행정동은 region_type == 'H'
    for r in rdocs:
        if r.get("region_type") == "H":
            return r.get("region_3depth_name")

    return None

hos["행정동"] = hos["전처리_도로명주소"].apply(addr_to_admin_dong)

hos.head()
hos.to_excel('행정동변환.xlsx')

In [21]:
#신설동, 용두동, 제기동, 전농1동, 전농2동, 답십리1동, 답십리2동, 장안1동, 장안2동, 청량리동, 회기동, 휘경1동, 휘경2동, 이문1동, 이문2동

#전농1동만 선택
new = hos[hos['행정동']=='전농1동']
new.shape

new.to_excel('전농1동추출.xlsx')

(58, 10)